In [1]:
from __future__ import print_function
import sdm as sdmlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
bits = 1000
sample = 1000000
radius = 451
scanner_type = sdmlib.SDM_SCANNER_OPENCL

In [3]:
address_space = sdmlib.AddressSpace.init_random(bits, sample)
counter = sdmlib.Counter.init_zero(bits, sample)
sdm = sdmlib.SDM(address_space, counter, radius, scanner_type)

In [4]:
def write_sequence(fold_list, bs_list):
    n = len(bs_list)
    for k in range(1, n+1):
        for i in range(n-k):
            print('Writing {} -> {} into sdm {}'.format(i, i+k, k-1))
            fold = fold_list[k-1]
            sdm.write(bs_list[i] ^ fold, bs_list[i+k])

In [5]:
bitstrings = {}
for x in ['A', 'B', 'C', 'D', 'E', 'F']:
    bitstrings[x] = sdmlib.Bitstring.init_random(bits)

In [6]:
folds = []
for _ in range(3):
    folds.append(sdmlib.Bitstring.init_random(bits))

In [7]:
def get_bs_sequence(seq):
    v = []
    for x in seq:
        v.append(bitstrings[x])
    return v

In [8]:
write_sequence(folds, get_bs_sequence('ABCD'))

Writing 0 -> 1 into sdm 0
Writing 1 -> 2 into sdm 0
Writing 2 -> 3 into sdm 0
Writing 0 -> 2 into sdm 1
Writing 1 -> 3 into sdm 1
Writing 0 -> 3 into sdm 2


In [9]:
write_sequence(folds, get_bs_sequence('EBCF'))

Writing 0 -> 1 into sdm 0
Writing 1 -> 2 into sdm 0
Writing 2 -> 3 into sdm 0
Writing 0 -> 2 into sdm 1
Writing 1 -> 3 into sdm 1
Writing 0 -> 3 into sdm 2


In [10]:
def read_sequence(fold_list, bs_list):
    n = len(bs_list)
    counters = []
    for i in range(n):
        print('Reading {} from sdm {}'.format(i, n-i-1))
        fold = fold_list[n-i-1]
        c = sdm.read_counter(bs_list[i] ^ fold)
        counters.append(c)
    final = sdmlib.Counter.init_zero(bits, 1)
    for c in counters:
        final.add_counter(0, c, 0)
    bs_final = final.to_bitstring(0)
    return bs_final

In [11]:
def calculate_distances(ref):
    v = [(x.distance_to(ref), key) for key, x in bitstrings.iteritems()]
    v.sort(key=lambda x: x[0])
    return v

In [12]:
bs_final = read_sequence(folds, get_bs_sequence('ABC'))

Reading 0 from sdm 2
Reading 1 from sdm 1
Reading 2 from sdm 0


In [13]:
for d, x in calculate_distances(bs_final):
    print('{:4d} {}'.format(d, x))

   0 D
 503 A
 503 C
 509 B
 512 F
 543 E


In [14]:
bs_final2 = read_sequence(folds, get_bs_sequence('EBC'))

Reading 0 from sdm 2
Reading 1 from sdm 1
Reading 2 from sdm 0


In [15]:
for d, x in calculate_distances(bs_final2):
    print('{:4d} {}'.format(d, x))

   0 F
 479 A
 507 E
 512 D
 517 C
 519 B
